<a href="https://colab.research.google.com/github/kamisoel/DigiGait/blob/main/Pipeline_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Setup environment
Install all necessary libraries and clone the github

In [8]:
%cd /content
%load_ext autoreload

!pip -q install -U pyyaml
!pip -q install yacs
!pip -q install mediapipe
!pip -q install slicerator
!pip -q install vg
#!pip -q install google_drive_downloader
!git clone https://github.com/kamisoel/gait-analyzer

import sys
if "gait-analyzer" not in sys.path:
    sys.path.append("gait-analyzer")


/content
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
fatal: destination path 'gait-analyzer' already exists and is not an empty directory.


Define some helper methods

In [9]:
import base64
from pathlib import Path
from google_drive_downloader import GoogleDriveDownloader as gdd
from IPython.display import HTML

import numpy as np
import vg
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import filtfilt, butter
from scipy.ndimage.filters import gaussian_filter1d

# often used joint ids for (17 joints) Human3.6m
MHip = 0
LHip, LKnee, LAnkle = 4,5,6
RHip, RKnee, RAnkle = 1,2,3


def show_local_mp4_video(file_name, width=640, height=480):
  """Define function to display the video"""

  video_encoded = base64.b64encode(Path(file_name).read_bytes()).decode()
  return HTML(data=f'''<video width="{width}" height="{height}" alt="test" controls>
                        <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"/>
                      </video>''')


def low_pass_filter(data, lp_freq=10, fs=50, order=4):
    """zero-lag butterworth low pass filter of given order using signal frequence fs"""

    nyq = 0.5 * fs
    b, a = butter(order, lp_freq/nyq, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)


def gauss_filter(data, sigma=5):
    return gaussian_filter1d(data, sigma, axis=0)


def calc_angle(pose, joint_idx):
    """calculate the angle between the given joints"""

    Hip, Knee, Ankle = joint_idx
    humerus = pose[:, Knee] - pose[:, Hip]
    tibia = pose[:, Knee] - pose[:, Ankle]
    return 180 - vg.angle(humerus, tibia)
    

def draw_skeleton(p, axis=0):
    plt.figure(figsize=(5,10))
    plt.scatter(p[:, axis], p[:, -1])
    for j in range(p.shape[0]):
        plt.text(p[j, axis], p[j, -1], str(j), color="red", fontsize=12)
    plt.show()


def norm_walking_dir(pose, joint_idx=[0, 1, 4]):
    MHip, RHip, LHip = joint_idx
    x_axis = np.array([1,0,0])
    z_axis = np.array([0,0,1])
    origin = pose[:, [MHip]]

    orient = vg.angle((pose[:, LHip] - pose[:, RHip]), x_axis, look=z_axis)
    new_pose = np.zeros_like(pose)
    for i in range(len(pose)):
        new_pose[i] = vg.rotate(pose[i] - origin[i], z_axis, orient[i])
    return new_pose + origin

#Create h36m 2d dataset for MediaPipe

Test videos of Human3.6m (S9, S11) should be given according to the following example folder structure: "{in_dir}/S9/Walking/Walking.55011271.mp4" </br>
Different trials (e.g. Walking and Walking 1) should have their own folder each

In [ ]:
in_dir = Path('/content/input')

In [ ]:
%cd ./gait-analyzer

from data.video import Video
from data.angle_helper import calc_common_angles
from data.data_utils import suggest_metadata
from model.mediapipe_estimator import MediaPipe_Estimator2D
from model.videopose3d import VideoPose3D

# map cam id to list index (from VideoPose3D)
cam_map = {
    '54138969': 0,
    '55011271': 1,
    '58860488': 2,
    '60457274': 3,
}

positions_2d = {}
positions_3d = {}
angles_3d = {}

# get appropiate meta data for MS coco pose topology
metadata = suggest_metadata('coco')

# Iterate over all subjects
for subj_dir in in_dir.iterdir():
    subject = subj_dir.name
    positions_2d[subject] = {}
    positions_3d[subject] = {}
    angles_3d[subject] = {}

    # Iterate over all activities
    for act_dir in subj_dir.iterdir():
        activity = act_dir.name
        positions_2d[subject][activity] = [None] * len(cam_map)
        positions_3d[subject][activity] = [None] * len(cam_map)
        angles_3d[subject][activity] = [None] * len(cam_map)

        # every file is by one camera
        for video_files in act_dir.iterdir():
            cam = video_files.stem.split('.')[1]
            video = Video(video_files)
  
            # estimate 2D and 3D keypoints using the HPE pipeline
            estimator_2d = MediaPipe_Estimator2D(out_format='coco')
            estimator_3d = VideoPose3D()

            kpts, meta = estimator_2d.estimate(video)
            pose_3d = estimator_3d.estimate(kpts, meta)['video']
            angles = calc_common_angles(pose_3d)

            # save data at correct list position
            id = cam_map[cam]
            pose_2d = kpts['video']['custom'][0]
            positions_2d[subject][activity][id] = pose_2d
            positions_3d[subject][activity][id] = pose_3d
            angles_3d[subject][activity][id] = angles

# save the data in VideoPose3D compatible npz format
np.savez_compressed("/content/data_2d_h36m_coco_mp", positions_2d=positions_2d,
                    positions_3d=positions_3d, angles_3d=angles_3d, metadata=metadata)

# Run 3D pose evaluation
The evaluation script of VideoPose3D will be used to evaluate the results

First VideoPose3D has to be setup by cloning the github and downloading the pretrained model and the gt data. The above calculated "data_2d_h36m_coco_mp.npz" should be copied in the VideoPose3D/data directory

In [ ]:
%cd ..
!git clone https://github.com/facebookresearch/VideoPose3D.git
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin -P VideoPose3D/checkpoint

from google_drive_downloader import GoogleDriveDownloader as gdd
# Download the gt 3D data => since redristribution of H36m is prohibited by 
# their licence, please see the VideoPose3D README for details on how to format
# the gt data for evaluation
#gdd.download_file_from_google_drive("1VmI39nv4k3DOgqDBQutoZaTl7bnFRtUv", 'VideoPose3D/data/data_3d_h36m.npz')

# copy the calculated 2D estimations to VideoPose3D/data directory
!cp data_2d_h36m_coco_mp.npz VideoPose3D/data/
%cd ./VideoPose3D

In [ ]:
## run this if the gt data contains all subjects data. Keeps only test subjects S9 and S11

pos_3d = np.load('data/data_3d_h36m.npz', allow_pickle=True)['positions_3d'].item()
test_pos_3d = {}

for s in ['S9', 'S11']:
    test_pos_3d[s] = {}
    for a in pos_3d[s]:
         test_pos_3d[s][a] = pos_3d[s][a]

np.savez_compressed('data/data_3d_h36m', positions_3d=test_pos_3d)

/content/VideoPose3D


Run VideoPose3D's evaluation script run.py using the estimated data

In [ ]:
!python run.py -k coco_mp -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin

/content/VideoPose3D
Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='h36m', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='coco_mp', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=False, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action=None, viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=5, viz_skip=0, viz_subject=None, viz_video=None, warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO: Trainable paramet

For comparision the original backbones for VideoPose3D can be downloaded and evaluated as well

In [ ]:
# Download other 2D backbones for comparision
!wget https://dl.fbaipublicfiles.com/video-pose-3d/data_2d_h36m_detectron_ft_h36m.npz -P data
!wget https://dl.fbaipublicfiles.com/video-pose-3d/data_2d_h36m_cpn_ft_h36m_dbb.npz -P data
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_cpn.bin -P checkpoint
#!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron.bin -P checkpoint

/content/VideoPose3D
--2021-08-13 09:30:48--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2021-08-13 09:30:48 ERROR 403: Forbidden.



In [ ]:
!python run.py -k detectron_ft_h36m -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='h36m', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_cpn.bin', export_training_curves=False, keypoints='detectron_ft_h36m', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=False, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action=None, viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=5, viz_skip=0, viz_subject=None, viz_video=None, warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO: Trainable parameter count: 16952371
Loa

In [ ]:
!python run.py -k cpn_ft_h36m_dbb -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='h36m', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_cpn.bin', export_training_curves=False, keypoints='cpn_ft_h36m_dbb', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=False, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action=None, viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=5, viz_skip=0, viz_subject=None, viz_video=None, warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO: Trainable parameter count: 16952371
Loadi

# Evaluate angle estimation

Setup needed data

In [ ]:
# folder containing the estimated 2d/3d data and the gt knee angle data
data_dir = Path('data')

# download gt angles
gdd.download_file_from_google_drive("1VmI39nv4k3DOgqDBQutoZaTl7bnFRtUv", data_dir / 'h36m_features.npz')

mp_data = np.load(data_dir / "data_2d_h36m_coco_mp.npz", allow_pickle=True)
mp_pos2d = mp_data['positions_2d'].item()
mp_pos3d = mp_data['positions_3d'].item()
mp_angles = mp_data['angles_3d'].item()
mp_meta = mp_data['metadata'].item()

gt_data = np.load(data_dir / "h36m_features.npz", allow_pickle=True)
#gt_pos3d = gt_data['positions_3d'].item()
gt_angles = gt_data['rotations_3d'].item()

# simplify skeleton to 17 joints
removed_joints = [4, 5, 9, 10, 11, 16, 20, 21, 22, 23, 24, 28, 29, 30, 31] # from VideoPose3D
kept_joints = [j for j in range(32) if j not in removed_joints]
for subject in gt_angles:
    for actions in gt_angles[subject]:
        #gt_pos3d[subject][actions] = gt_pos3d[subject][actions][:, kept_joints]
        gt_angles[subject][actions] = gt_angles[subject][actions][:, kept_joints]

In [ ]:
from model.videopose3d import VideoPose3D
from sklearn.metrics import explained_variance_score

def mae_error(A, B):
    return np.abs(A - B).mean(axis=0)

def msd_error(A, B):
    return (A - B).mean(axis=0)

def rmse_error(A, B):
    return np.sqrt(np.square(A - B).mean(axis=0))

def corrcoef(A, B):
    if A.ndim >1 and A.shape[-1] > 1:
        return np.diag(np.corrcoef(A, B, rowvar=False), k=A.shape[-1])
    else:
        return np.corrcoef(A, B, rowvar=False)[1,0]


def evaluate(pos2d, gt_angles, **kwargs):
    results = []
    if 'normalized_skeleton' in kwargs:
        videopose_estimator = VideoPose3D(normalized_skeleton=kwargs['normalized_skeleton'])
    else:
        videopose_estimator = VideoPose3D()
    for subject in ['S9', 'S11']:
        for action in ['Walking', 'Walking 1', 'WalkTogether', 'WalkTogether 1']: #mp_pos2d[subject].keys():
            for cam in range(len(mp_pos2d[subject][action])):
                gt = np.rad2deg(gt_angles[subject][action])
                p_2d = pos2d[subject][action][cam]

                if 'pre_filter' in kwargs:
                    if kwargs['pre_filter'] == 'gauss':
                        sigma = kwargs['sigma'] if 'sigma' in kwargs else 5
                        p_2d = gauss_filter(p_2d, sigma)
                    elif kwargs['pre_filter'] == 'butter':
                        freq = kwargs['freq'] if 'freq' in kwargs else 20
                        p_2d = low_pass_filter(p_2d, freq)

                p_3d_est = videopose_estimator.estimate({'video': {'custom': [p_2d]}}, mp_meta)
                p_3d = p_3d_est['video'][:len(gt)]
                if 'skip_frames' in kwargs:
                    p_3d = p_3d[kwargs['skip_frames']:]
                    gt = gt[kwargs['skip_frames']:]

                if 'post_filter' in kwargs:
                    if kwargs['post_filter'] == 'gauss':
                        sigma = kwargs['sigma'] if 'sigma' in kwargs else 5
                        p_3d = gauss_filter(p_3d, sigma)
                    elif kwargs['post_filter'] == 'butter':
                        freq = kwargs['freq'] if 'freq' in kwargs else 20
                        p_3d = low_pass_filter(p_3d, freq)

                rknee = calc_angle(p_3d, [RHip, RKnee, RAnkle])
                lknee = calc_angle(p_3d, [LHip, LKnee, LAnkle])
                scale = kwargs['scale_factor'] if 'scale_factor' in kwargs else 1
                knees = scale * np.stack([rknee, lknee], axis=1)
                gt_knees = gt[:, [RKnee, LKnee], 0]

                mae = mae_error(gt_knees, knees)
                rmse = rmse_error(gt_knees, knees)
                msd = msd_error(gt_knees, knees)
                corr = corrcoef(gt_knees, knees)
                vaf = explained_variance_score(gt_knees, knees)

                action_split = action.split(' ')
                action_class = action_split[0]
                trial = 1+int(action_split[1]) if len(action_split)>1 else 1
                results.append(dict(subject=subject, action=action_class, trial=trial, cam=cam, 
                                    mae=np.mean(mae), rmse=np.mean(rmse), msd=np.mean(msd),
                                    corr_coeff=np.mean(corr), vaf=vaf,
                                    r_mae=mae[0], l_mae=mae[1],
                                    r_rmse=rmse[0], l_rmse=rmse[1],
                                    r_msd=msd[0], l_msd=msd[1],
                                    r_corr_coeff=corr[0], l_corr_coeff=corr[1]),)
    results = pd.DataFrame(results)
    results['subject'] = results['subject'].astype("category") # wont get aggregated
    results['action'] = results['action'].astype("category")
    results['trial'] = results['trial'].astype("category")
    results['cam'] = results['cam'].astype("category")
    return results

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

results = evaluate(mp_pos2d, gt_angles)
results_scale = evaluate(mp_pos2d, gt_angles, scale_factor=1.1)
results_scale2 = evaluate(mp_pos2d, gt_angles, scale_factor=1.2)
results_scale3 = evaluate(mp_pos2d, gt_angles, scale_factor=1.25)
results_butter_pre = evaluate(mp_pos2d, gt_angles, pre_filter='butter', freq=6)
results_butter_post = evaluate(mp_pos2d, gt_angles, post_filter='butter', freq=6)
results_butter_pre_scale = evaluate(mp_pos2d, gt_angles, pre_filter='butter', freq=6, scale_factor=1.2)
results_butter_post_scale = evaluate(mp_pos2d, gt_angles, post_filter='butter', freq=6, scale_factor=1.2)

final_results = pd.concat([results, results_scale, results_scale2, results_scale3,
                     results_butter_pre, results_butter_post, results_butter_pre_scale,
                     results_butter_post_scale],
                     names=['condition'],
                     keys=['Original', 'Scale 1.1', 'Scale 1.2', 'Scale 1.25', 
                           'Butter Pre', 'Butter Post', 'Butter Pre + Scale', 'Butter Post + Scale'])

#results.groupby('action').agg(['mean', 'std', 'min', 'max'])
final_results.groupby('condition').agg(['mean', 'std', 'min', 'max'])

#Evaluation for nonlinear metrics
(the data setup cell from the angle evaluation has to be run first)

In [ ]:
!pip install -q nolds
import nolds

def evaluate_nl(pos2d, gt_angles, **kwargs):
    results = []
    if 'normalized_skeleton' in kwargs:
        videopose_estimator = VideoPose3D(normalized_skeleton=kwargs['normalized_skeleton'])
    else:
        videopose_estimator = VideoPose3D()
    for subject in ['S9', 'S11']:
        for action in ['Walking']:#, 'Walking 1', 'WalkTogether', 'WalkTogether 1']: #mp_pos2d[subject].keys():
            for cam in range(len(mp_pos2d[subject][action])):
                gt = np.rad2deg(gt_angles[subject][action])
                p_2d = pos2d[subject][action][cam]
                #3d estimation
                p_3d_est = videopose_estimator.estimate({'video': {'custom': [p_2d]}}, mp_meta)
                p_3d = low_pass_filter(p_3d_est['video'][:len(gt)], 6)
                # knee angles
                rknee = calc_angle(p_3d, [RHip, RKnee, RAnkle])
                lknee = calc_angle(p_3d, [LHip, LKnee, LAnkle])
                scale = kwargs['scale_factor'] if 'scale_factor' in kwargs else 1
                knees = scale * np.stack([rknee, lknee], axis=1)
                gt_knees = gt[:, [RKnee, LKnee], 0]
                # metadata
                action_split = action.split(' ')
                action_class = action_split[0]
                trial = 1+int(action_split[1]) if len(action_split)>1 else 1
                for side in [0,1]: #right, left
                    # nl metrics
                    lyap = nolds.lyap_r(knees[:, side], 3, 4, 50)
                    entropy = nolds.sampen(knees[:, side], 3)
                    corr_dim = nolds.corr_dim(knees[:, side], 3)
                    gt_lyap = nolds.lyap_r(gt_knees[:, side], 3, 4, 50)
                    gt_entropy = nolds.sampen(gt_knees[:, side], 3)
                    gt_corr_dim = nolds.corr_dim(gt_knees[:, side], 3)

                    results.append(dict(subject=subject, action=action_class, 
                                        trial=trial, cam=cam, side=side,
                                        lyap=lyap, gt_lyap=gt_lyap,
                                        entropy=entropy, gt_entropy=gt_entropy,
                                        corr_dim=corr_dim, gt_corr_dim=gt_corr_dim
                                        ))
    results = pd.DataFrame(results)
    results['subject'] = results['subject'].astype("category") # wont get aggregated
    results['action'] = results['action'].astype("category")
    results['trial'] = results['trial'].astype("category")
    results['cam'] = results['cam'].astype("category")
    return results

In [ ]:
evaluate_nl(mp_pos2d, gt_angles, scale_factor=1.2)

# Evaluate gait event detection
The manuell event time annotation can be downloaded from [google drive](https://drive.google.com/file/d/17rqD6vrUlnYRzbQaitzz5FQ2m9m1ZziF/view?usp=sharing) as json files. The trimmed and interpolated videos cannot be provided but the original files can be found on the [MissionGait Website](https://www.missiongait.org/case-study-videos). An example for proessing using ffmpeg will be given. Pre-calculated keypoint locations and knee angles are provided, but can be also be computed yourself using the processed videos.

In [1]:
MG_DATA = {'13_hemiplegic_gait.npz': '1bgFQzuXCGMQxK5VaL2u22GOsqAt8N60C',
           '17_chronic_hemiparetic_gait.npz': '1R9SLU-e-33eC208nIhbZ9fusQ8CdbeCm',
           '24_gait_dystonia.npz': '11UM90QPa61X1OBv60h78fc_DugUR_3r0',
           '31_leg_length_discrepancy.npz': '16JXwuXoqtrQSg6CSPj3daRDIoFGA2vLn',
           '32_diabetic_neuropathy.npz': '1w0Da4n2h9k-7tkKjvFW0Hz49IPq81w0l'}

EVENT_ANNOT = "17rqD6vrUlnYRzbQaitzz5FQ2m9m1ZziF"

def download_pre_calculated(out_dir):
    for name, glink in MG_DATA.items():
        gdd.download_file_from_google_drive(glink, Path(out_dir) / name)


def download_event_annots(out_dir):
    gdd.download_file_from_google_drive(EVENT_ANNOT, 
                                        Path(out_dir) / 'events.zip', 
                                        unzip=True)
    !rm {out_dir}/events.zip   


def estimate_from_videos(path):
    results = {}
    est_2d = MediaPipe_Estimator2D(out_format='coco')
    est_3d = VideoPose3D()
    for p in Path(path).glob('*.mp4'):
        print(f"Estimating keypoints for video {p.name}")
        video = Video(p)
        # set length according to walking from left to right
        if p.stem.startswith('24'):
            video = video[:330]
        elif p.stem.startswith('31'):
            video = video[:350]
        else:
            video = video[:500]
        # estimate 2D and 3D keypoints and calculate knee angles
        kpts, meta = est_2d.estimate(video)
        pose_2d = kpts['video']['custom'][0]
        pose_3d = est_3d.estimate(kpts, meta)
        pose_3d = low_pass_filter(norm_walking_dir(pose_3d['video']))
        rknee = calc_angle(pose_3d, [RHip, RKnee, RAnkle])
        lknee = calc_angle(pose_3d, [LHip, LKnee, LAnkle])
        angles = np.stack([rknee, lknee], axis=-1)
        results[p.stem] = dict(pose_2d=pose_2d, pose_3d=pose_3d, angles=angles)
    return results


def read_event_annots(event_dir):
    import json
    events = {}
    for p in Path(event_dir).iterdir():
        if p.stem.startswith('.'):
            continue
        event_file = p / 'events.json'
        event_data = json.loads(event_file.read_text())
        events[p.stem] = [np.array(event_data[e]) for e in ['RHS', 'LHS', 'RTO', 'LTO']]
    return events


def create_dataset(out_dir, video_dir='videos', event_dir='events'):
    download_event_annots(event_dir)
    events = read_event_annots(event_dir)

    video_data = estimate_from_videos(video_dir)

    for k, data in videos.items():
        print('Saving data:', k)
        np.savez_compressed(Path(out_dir) / k, pose_2d = data['pose_2d'], 
                            pose_3d = data['pose_3d'], angles = data['angles'], 
                            events=events[k])

In [16]:
from data.gait_cycle_detector import GaitCycleDetector as GCD
from data.timeseries_utils import align_values

def eval_alg(detect_mode, pose_3d, targets):
    gcd = GCD()

    yhat = gcd.detect(pose_3d, detect_mode)
    if detect_mode == 'hhd':
        yhat = yhat[:2]

    results = []
    for i, event in enumerate(yhat):
        fp = 0
        fn = 0
        diffs = []
        comp = align_values(targets[i], event, f='zip', tolerance=15, keep='both')
        for c in comp:
          if np.isnan(c[0]):
              fp += 1
          elif np.isnan(c[1]):
              fn += 1
          else:
              diffs.append((c[1] - c[0]))

        event = 'HS' if i < 2 else 'TO'
        side = 'right' if i % 2 == 0 else 'left'
        mae = sum([abs(v) for v in diffs]) / len(diffs) if len(diffs) > 0 else np.NaN
        msd = sum(diffs) / len(diffs) if len(diffs) > 0 else np.NaN

        results.append((event, side, len(targets[i]), len(targets[i]) + fp - fn, fp, fn, 
                        mae, msd))
    return pd.DataFrame(results, columns=['Event', 'Side', 'Total','Detected', 'False Pos',
                                          'False Neg', 'MAE', 'MSD']).set_index(['Event', 'Side'])
        

def eval_all(video_folder):
    ALG_NAMES = ['auto', 'fva', 'hhd', 'rfd']
    ALG_NAMES_VERBOSE = ['Combined', 'Foot Velocity Alg.', 
                         'Horizontal Heel Displacement', 'Foot Displacement']
    
    results = []
    ns = []
    for p in Path(video_folder).iterdir():
        if p.stem.startswith('.'):
            continue
        data = np.load(p, allow_pickle=True)
        video = p.stem
        pose_2d = data['pose_2d']
        pose_3d = data['pose_3d']
        angles = data['angles']
        events = list(data['events'])

        for i, alg in enumerate(ALG_NAMES):
            ns.append((video, ALG_NAMES_VERBOSE[i]))
            results.append(eval_alg(alg, pose_3d, events))

    return pd.concat(results, keys=ns, names=['Video', 'Algorithm'])

In [17]:
DATA_DIR = 'eval_data'

if not Path(DATA_DIR).is_dir() or not any(Path(DATA_DIR).iterdir()):
    #create_dataset(DATA_DIR, VIDEO_DIR, EVENT_DIR)
    download_pre_calculated(DATA_DIR)

FPS = 50
results = eval_all(DATA_DIR)

pd.options.display.float_format = '{:,.2f}'.format
display(results.groupby(['Algorithm', 'Event'])[['Total', 'Detected', 'False Pos', 'False Neg']].agg(['sum']))
display(results.groupby(['Algorithm', 'Event'])[['MAE', 'MSD']].agg(['mean', 'std']) * FPS)

Total Detected False Pos False Neg
                                     sum      sum       sum       sum
Algorithm                    Event                                   
Combined                     HS       60       58         0         2
                             TO       60       62         2         0
Foot Displacement            HS       60       58         0         2
                             TO       60       62         2         0
Foot Velocity Alg.           HS       60       67        12         5
                             TO       60       59         3         4
Horizontal Heel Displacement HS       60       83        24         1

MAE            MSD       
                                     mean    std    mean    std
Algorithm                    Event                             
Combined                     HS     91.24  78.73  -40.81  96.74
                             TO    205.14 102.82 -205.14 102.82
Foot Displacement            HS    159.40  86.73 -154.40  94.31
                             TO    190.64  90.52 -190.64  90.52
Foot Velocity Alg.           HS    338.83 157.00  334.83 165.46
                             TO    198.67  67.39 -198.67  67.39
Horizontal Heel Displacement HS    131.76 134.84   88.76 165.97